<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE__1D_CNN_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow_io

In [2]:
import os
import time
import json
import datetime

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_io as tfio

from sklearn.metrics import classification_report, confusion_matrix

tf.random.set_seed(42)

tf.__version__

'2.4.1'

In [3]:
CONFIG = {
    'timestamp'             : str(datetime.datetime.now()),
    'model'                 : 'Transfer Learning on YAMNET',
    'test_fold'             : 'Fold 5',
    'Fs'                    : 16000,
    'frame_length'          : 16000,
    'frame_step'            : 8000,
    'architecture'          : '',
    'batch_size'            : 32,
    'epochs'                : 0,
    'learning_rate'         : 0.000005,
    'monitor'               : 'val_loss',
    'patience'              : 3,
    'class_weight'          : { 0: 0.71, 1:  2.32, 2: 0.86 },
    'training_time'         : 0,
    'testing_time'          : 0,
    'cm_atick'              : '',
    'cr_atick'              : '',
    'cm_valentina'          : '',
    'cr_valentina'          : ''
}

In [4]:

BASE_DIR            = os.getcwd()
LOG_FILE            = '/content/drive/MyDrive/Research/Crowd Emotion Logs/tl_yamnet.txt'
FOLDS               = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS            = ['Approval', 'Disapproval', 'Neutral']

ATICK_DATA_DIR      = '/content/Dataset-Atick/'
VALENTINA_DATA_DIR  = '/content/Dataset-Valentina/'


In [5]:
%%time

# # ----------------- Loading my dataset -------------------
# !mkdir /content/Dataset-Atick/
# !gdown --id '1HOe5sJe_Juf5uib4f-8pPv-Z64lmqQ4X'
# !tar -xf /content/Crowd-Emotion-Dataset.tar.xz -C /content/Dataset-Atick/

# # ----------------- Loading Valentina's dataset -------------------
# !mkdir /content/Dataset-Valentina/
# !gdown --id '11tC2Nmie9v3ljo60oQJ3sN1rVkorV-N1'
# !tar -xf /content/Valentina_CE_Dataset.tar.xz -C /content/Dataset-Valentina/


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [6]:
# --------------- File loading utilities ------------------

@tf.function
def load_wav_mono(filename):
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=CONFIG['Fs'])
    return wav

def load_wav_for_map(filename, label, fold):
  return load_wav_mono(filename), label, fold

def load_ds(path, class_names, fold_names=['']):

    filenames = []
    labels = []
    folds = []

    for class_name in class_names:
        print('processing files for ' + class_name, end=' ... ')

        for fold in fold_names:
            files_path = os.path.join(path, class_name, fold)

            for filename in os.listdir(files_path):
                filenames.append(os.path.join(files_path, filename))
                labels.append(class_names.index(class_name))

                try:
                    folds.append(FOLDS.index(fold))
                except ValueError:
                    folds.append(0)

        print('√')

    return tf.data.Dataset.from_tensor_slices((filenames, labels, folds))

In [22]:
atick_ds = load_ds(ATICK_DATA_DIR, EMOTIONS, FOLDS)
valentina_ds = load_ds(VALENTINA_DATA_DIR, EMOTIONS)

# atick_ds.element_spec

processing files for Approval ... √
processing files for Disapproval ... √
processing files for Neutral ... √
processing files for Approval ... √
processing files for Disapproval ... √
processing files for Neutral ... √


In [23]:
atick_ds = atick_ds.map(load_wav_for_map)
valentina_ds = valentina_ds.map(load_wav_for_map)

In [9]:
def get_fft_coeffs(wav):
    wav = tf.cast(wav, tf.float32)
    fft = tf.signal.rfft(wav)
    return tf.abs(fft)

def frames_map(wav, label, fold):
    frames = tf.signal.frame(wav, CONFIG['frame_length'], CONFIG['frame_step'], axis=0)
    num_frames = tf.shape(frames)[0]
    return (
        frames,
        tf.repeat(label, num_frames),
        tf.repeat(fold, num_frames)
    )

def fft_map(frame, label, fold):
    return get_fft_coeffs(frame), label, frame


In [24]:
atick_ds = atick_ds.map(frames_map).unbatch()
valentina_ds = valentina_ds.map(frames_map).unbatch()

atick_ds.element_spec

(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None))

In [25]:
atick_ds = atick_ds.map(fft_map)
valentina_ds = valentina_ds.map(fft_map)

atick_ds.element_spec

(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None),
 TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))

In [26]:
cached_ds_atick = atick_ds.cache()
cached_ds_valentina = valentina_ds.cache()

train_ds = cached_ds_atick.filter(
    lambda frame, label, fold: fold != FOLDS.index(CONFIG['test_fold'])
)
val_ds = cached_ds_atick.filter(
    lambda frame, label, fold: fold == FOLDS.index(CONFIG['test_fold'])
)

test_ds = cached_ds_valentina

# train_ds = cached_ds_atick
# val_ds = cached_ds_valentina
# test_ds = cached_ds_valentina

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda frame, label, fold: (frame, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000) .batch(CONFIG['batch_size']) \
                    .prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(CONFIG['batch_size']).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(CONFIG['batch_size']).prefetch(tf.data.AUTOTUNE)

In [27]:
for fft, _, _ in atick_ds.take(1):
    input_shape = fft.shape[0]

print(input_shape)

8001


In [14]:
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(atick_ds.map(lambda x, label, fold: x))


In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(input_shape, 1)),
    norm_layer,
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(EMOTIONS))
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 8001, 1)           3         
_________________________________________________________________
conv1d (Conv1D)              (None, 7999, 64)          256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7997, 64)          12352     
_________________________________________________________________
dropout (Dropout)            (None, 7997, 64)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3998, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 255872)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [16]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
    metrics=['accuracy']
)

callback = tf.keras.callbacks.EarlyStopping(
    monitor=CONFIG['monitor'],
    patience=CONFIG['patience'],
    restore_best_weights=True
)

In [28]:
%%time
start_time = time.time()

history = model.fit(
    train_ds,
    epochs              = 300,
    validation_data     = val_ds,
    callbacks           = callback,
    class_weight        = CONFIG['class_weight']
)

training_time = time.time() - start_time

Epoch 1/300


InvalidArgumentError: ignored